![FunMAn Overview](./funman-diagram.png)

- # FUNMAN Approach:
  - Identify regions of parameter space that satisfy operational constraints
  - Compile model and operational constraints into first-order logic w/ theory of nonlinear real arithmetic (Satisfiabily Modulo Theory, SMT)
  - Solve series of SMT instances (proving unsatisfiability) to label regions.  Two subproblems: 1) identify a region to check, 2) determine how to check
- # Phase Goals: 
  - Improve Scalability
  - Expand Models 
  - Feedback to TA1 and TA3
- # Status: 
  - Scalability: MCTS in dReal solver (exponential speedup in precision) (subproblem 2, above)
  - Models: Petrinet, Regnet, Bilayers (multiple formats)
  - API: Submit Request, Get results anytime, Halt job
  - Integration: building on Terrarium, ready for deployment and TA3 usage
- # Ongoing Work: 
  - Variable Elimination in SMT Encodings (subproblem 2, above)
  - Exists Forall encodings (subproblem 1, above)
  - API support for new models and operational constraints


# Demo Setup

In [ ]:
# Common imports
import asyncio
import nest_asyncio
import os
import json
from pathlib import Path

from IPython.display import JSON

# Setup URLs
API_BASE_PATH = os.path.join(os.getcwd(), "..")
# Currently assumes a localhost server us running (see funman_server.ipynb)
API_SERVER_HOST = "127.0.0.1"
API_SERVER_PORT = 8190
SERVER_URL = f"http://{API_SERVER_HOST}:{API_SERVER_PORT}"
OPENAPI_URL = f"{SERVER_URL}/openapi.json"
CLIENT_NAME = "funman-api-client"

# Setup Paths
RESOURCES = Path("../resources").resolve()


# Install API client to the working directory
# and add the install to the sys path
import funman.api.client as client
client.make_client(
    API_BASE_PATH, openapi_url=OPENAPI_URL, client_name=CLIENT_NAME
)
nest_asyncio.apply()


# Setup API client request
from funman_api_client import Client
from funman_api_client.api.default import (
    post_queries_queries_post,
    get_queries_queries_query_id_get,
)
from funman_api_client.models.body_post_queries_queries_post import BodyPostQueriesQueriesPost
from funman_api_client.models import (
    BilayerModel,
    FUNMANConfig,
    LabeledParameter,
    QueryLE,
    FunmanWorkRequest,
    FunmanWorkUnit,
    FunmanResults,
    QueryTrue,
    GeneratedPetriNetModel
    
)

# Create a funman client
funman_client = Client(SERVER_URL, timeout=None)

In [ ]:
job="6bef43a3-27cf-41b0-a132-dba680800657"

# Make a GET request to the API endpoint
response = asyncio.run(get_queries_queries_query_id_get.asyncio_detailed(
    job,
    client=funman_client
))
# The response returns the latest funman results for the query
results = FunmanResults.from_dict(
    src_dict=json.loads(response.content.decode())
)
# pretty print the parameter_space
print(json.dumps(results.parameter_space.to_dict(), indent=2))

# Plot the ParameterSpace
from funman_demo.parameter_space_plotter import ParameterSpacePlotter
ParameterSpacePlotter(results.parameter_space).plot(show=True)

---
---
# Example 1: AMR Petrinet
## Question: Can we verify the parameter bounds given in the AMR prevent > 30% maximum infections?
---
## Example 1: Step 1: Load Model and Request

In [ ]:
EXAMPLE_DIR = RESOURCES / "amr" / "petrinet" / "amr-examples"
MODEL_PATH = EXAMPLE_DIR / "sir.json"
REQUEST_PATH = EXAMPLE_DIR / "sir_request1.json"

# Read in the model file
model = json.loads(MODEL_PATH.read_bytes())
# pretty print
# print(json.dumps(model, indent=2))

# Read in the funman request file
request = json.loads(REQUEST_PATH.read_bytes())
# pretty print
print(json.dumps(request, indent=2))

## Example 1: Step 2: Submit job

In [ ]:
# Make a POST request to the API endpoint
response = asyncio.run(
    post_queries_queries_post.asyncio_detailed(
        client=funman_client,
        json_body=BodyPostQueriesQueriesPost.from_dict({
            "model": model,
            "request": request
        }),
    )
)

# The response returns a work unit
work_unit = FunmanWorkUnit.from_dict(
    src_dict=json.loads(response.content.decode())
)
# Where the id can be used to pull ongoing results
work_unit.id

## Example 1: Step 3: Request results

In [ ]:
# Make a GET request to the API endpoint
response = asyncio.run(get_queries_queries_query_id_get.asyncio_detailed(
    work_unit.id,
    client=funman_client
))
# The response returns the latest funman results for the query
results = FunmanResults.from_dict(
    src_dict=json.loads(response.content.decode())
)
# pretty print the parameter_space
print(json.dumps(results.parameter_space.to_dict(), indent=2))

## Example 1: Step 4: Plot Results

In [ ]:
# Plot the ParameterSpace
from funman_demo.parameter_space_plotter import ParameterSpacePlotter
ParameterSpacePlotter(results.parameter_space).plot(show=True)

# Example 1b: Optional: Halt Processing Job

In [ ]:
# Halt the current run
from funman_api_client.api.default import (
    halt_queries_query_id_halt_get
)

response = asyncio.run(
    halt_queries_query_id_halt_get.asyncio_detailed(
        client=funman_client,
        query_id=work_unit.id
    )
)

# The response returns a status
response

---
---
# Example 1: AMR Petrinet
## Question: Can we verify the parameter bounds given in the AMR prevent > 30% maximum infections?
---
## Example 1a: Step 1: Load Model and Request

In [ ]:
EXAMPLE_DIR = RESOURCES / "amr" / "petrinet" / "amr-examples"
MODEL_PATH = EXAMPLE_DIR / "sir.json"
REQUEST_PATH = EXAMPLE_DIR / "sir_request1a.json"

# Read in the model file
model = json.loads(MODEL_PATH.read_bytes())
# pretty print
# print(json.dumps(model, indent=2))

# Read in the funman request file
request = json.loads(REQUEST_PATH.read_bytes())
# pretty print
print(json.dumps(request, indent=2))

## Example 1a: Step 2: Submit job

In [ ]:
# Make a POST request to the API endpoint
response = asyncio.run(
    post_queries_queries_post.asyncio_detailed(
        client=funman_client,
        json_body=BodyPostQueriesQueriesPost.from_dict({
            "model": model,
            "request": request
        }),
    )
)

# The response returns a work unit
work_unit = FunmanWorkUnit.from_dict(
    src_dict=json.loads(response.content.decode())
)
# Where the id can be used to pull ongoing results
work_unit.id

## Example 1a: Step 3: Request results

In [ ]:
# Make a GET request to the API endpoint
response = asyncio.run(get_queries_queries_query_id_get.asyncio_detailed(
    work_unit.id,
    client=funman_client
))
# The response returns the latest funman results for the query
results = FunmanResults.from_dict(
    src_dict=json.loads(response.content.decode())
)
# pretty print the parameter_space
print(json.dumps(results.parameter_space.to_dict(), indent=2))

## Example 1a: Step 4: Plot Results

In [ ]:
# Plot the ParameterSpace
from funman_demo.parameter_space_plotter import ParameterSpacePlotter
ParameterSpacePlotter(results.parameter_space).plot(show=True)

# Example 1a: Optional: Halt Processing Job

In [ ]:
# Halt the current run
from funman_api_client.api.default import (
    halt_queries_query_id_halt_get
)

response = asyncio.run(
    halt_queries_query_id_halt_get.asyncio_detailed(
        client=funman_client,
        query_id=work_unit.id
    )
)

# The response returns a status
response

---
---
# Example 1b: AMR Petrinet
## Question: Can we push beta up and gamma down?
---
## Example 1b: Step 1: Load Model and Request

In [ ]:
EXAMPLE_DIR = RESOURCES / "amr" / "petrinet" / "amr-examples"
MODEL_PATH = EXAMPLE_DIR / "sir.json"
REQUEST_PATH = EXAMPLE_DIR / "sir_request1b.json"

# Read in the model file
model = json.loads(MODEL_PATH.read_bytes())
# pretty print
# print(json.dumps(model, indent=2))

# Read in the funman request file
request = json.loads(REQUEST_PATH.read_bytes())
# pretty print
print(json.dumps(request, indent=2))

## Example 1b: Step 2: Submit job

In [ ]:
# Make a POST request to the API endpoint
response = asyncio.run(
    post_queries_queries_post.asyncio_detailed(
        client=funman_client,
        json_body=BodyPostQueriesQueriesPost.from_dict({
            "model": model,
            "request": request
        }),
    )
)

# The response returns a work unit
work_unit = FunmanWorkUnit.from_dict(
    src_dict=json.loads(response.content.decode())
)
# Where the id can be used to pull ongoing results
work_unit.id

## Example 1b: Step 3: Request results

In [ ]:
# Make a GET request to the API endpoint
response = asyncio.run(get_queries_queries_query_id_get.asyncio_detailed(
    work_unit.id,
    client=funman_client
))
# The response returns the latest funman results for the query
results = FunmanResults.from_dict(
    src_dict=json.loads(response.content.decode())
)
# pretty print the parameter_space
print(json.dumps(results.parameter_space.to_dict(), indent=2))

## Example 1b: Step 4: Plot Results

In [ ]:
# Plot the ParameterSpace
from funman_demo.parameter_space_plotter import ParameterSpacePlotter
ParameterSpacePlotter(results.parameter_space).plot(show=True)

# Example 1b: Optional: Halt Processing Job

In [ ]:
# Halt the current run
from funman_api_client.api.default import (
    halt_queries_query_id_halt_get
)

response = asyncio.run(
    halt_queries_query_id_halt_get.asyncio_detailed(
        client=funman_client,
        query_id=work_unit.id
    )
)

# The response returns a status
response

---
---
# Example 1c: Skema AMR Petrinet
## Question: Can we identify relevant parameter values when no ranges are extracted?
---
## Example 1c: Step 1: Load Model and Request

In [ ]:
EXAMPLE_DIR = RESOURCES / "amr" / "petrinet" / "skema"
MODEL_PATH = EXAMPLE_DIR / "linked_petrinet.json"
REQUEST_PATH = EXAMPLE_DIR / "sir_request_skema1c.json"

# Read in the model file
model = json.loads(MODEL_PATH.read_bytes())
# pretty print
# print(json.dumps(model, indent=2))

# Read in the funman request file
request = json.loads(REQUEST_PATH.read_bytes())
# pretty print
print(json.dumps(request, indent=2))

## Example 1c: Step 2: Submit job

In [ ]:
# Make a POST request to the API endpoint
response = asyncio.run(
    post_queries_queries_post.asyncio_detailed(
        client=funman_client,
        json_body=BodyPostQueriesQueriesPost.from_dict({
            "model": model,
            "request": request
        }),
    )
)

# The response returns a work unit
work_unit = FunmanWorkUnit.from_dict(
    src_dict=json.loads(response.content.decode())
)
# Where the id can be used to pull ongoing results
work_unit.id

## Example 1c: Step 3: Request results

In [ ]:
# Make a GET request to the API endpoint
response = asyncio.run(get_queries_queries_query_id_get.asyncio_detailed(
    work_unit.id,
    client=funman_client
))
# The response returns the latest funman results for the query
results = FunmanResults.from_dict(
    src_dict=json.loads(response.content.decode())
)
# pretty print the parameter_space
print(json.dumps(results.parameter_space.to_dict(), indent=2))

## Example 1c: Step 4: Plot Results

In [ ]:
# Plot the ParameterSpace
from funman_demo.parameter_space_plotter import ParameterSpacePlotter
ParameterSpacePlotter(results.parameter_space, plot_points=True).plot(show=True)

# Example 1c: Optional: Halt Processing Job

In [ ]:
# Halt the current run
from funman_api_client.api.default import (
    halt_queries_query_id_halt_get
)

response = asyncio.run(
    halt_queries_query_id_halt_get.asyncio_detailed(
        client=funman_client,
        query_id=work_unit.id
    )
)

# The response returns a status
response

---
---
# Example 1d: Skema AMR Petrinet
## Question: Can we identify all parameter values when no ranges are extracted?
---
## Example 1d: Step 1: Load Model and Request

In [ ]:
EXAMPLE_DIR = RESOURCES / "amr" / "petrinet" / "skema"
MODEL_PATH = EXAMPLE_DIR / "linked_petrinet.json"
REQUEST_PATH = EXAMPLE_DIR / "sir_request_skema1d.json"

# Read in the model file
model = json.loads(MODEL_PATH.read_bytes())
# pretty print
# print(json.dumps(model, indent=2))

# Read in the funman request file
request = json.loads(REQUEST_PATH.read_bytes())
# pretty print
print(json.dumps(request, indent=2))

## Example 1d: Step 2: Submit job

In [ ]:
# Make a POST request to the API endpoint
response = asyncio.run(
    post_queries_queries_post.asyncio_detailed(
        client=funman_client,
        json_body=BodyPostQueriesQueriesPost.from_dict({
            "model": model,
            "request": request
        }),
    )
)

# The response returns a work unit
work_unit = FunmanWorkUnit.from_dict(
    src_dict=json.loads(response.content.decode())
)
# Where the id can be used to pull ongoing results
work_unit.id

## Example 1d: Step 3: Request results

In [ ]:
# Make a GET request to the API endpoint
response = asyncio.run(get_queries_queries_query_id_get.asyncio_detailed(
    work_unit.id,
    client=funman_client
))
# The response returns the latest funman results for the query
results = FunmanResults.from_dict(
    src_dict=json.loads(response.content.decode())
)
# pretty print the parameter_space
print(json.dumps(results.parameter_space.to_dict(), indent=2))

## Example 1d: Step 4: Plot Results

In [ ]:
# Plot the ParameterSpace
from funman_demo.parameter_space_plotter import ParameterSpacePlotter
ParameterSpacePlotter(results.parameter_space, plot_points=True).plot(show=True)

# Example 1d: Optional: Halt Processing Job

In [ ]:
# Halt the current run
from funman_api_client.api.default import (
    halt_queries_query_id_halt_get
)

response = asyncio.run(
    halt_queries_query_id_halt_get.asyncio_detailed(
        client=funman_client,
        query_id=work_unit.id
    )
)

# The response returns a status
response

---
---
# Example 2: AMR Regnet
---
## Example 2: Step 1: Load Model and Request

In [ ]:
EXAMPLE_DIR = RESOURCES / "amr" / "regnet" / "amr-examples"
MODEL_PATH = EXAMPLE_DIR / "lotka_volterra.json"
REQUEST_PATH = EXAMPLE_DIR / "lotka_volterra_request1.json"

# Read in the model file
model = json.loads(MODEL_PATH.read_bytes())
# pretty print
# print(json.dumps(model, indent=2))

# Read in the funman request file
request = json.loads(REQUEST_PATH.read_bytes())
# pretty print
print(json.dumps(request, indent=2))

## Example 2: Step 2: Submit job

In [ ]:
# Make a POST request to the API endpoint
response = asyncio.run(
    post_queries_queries_post.asyncio_detailed(
        client=funman_client,
        json_body=BodyPostQueriesQueriesPost.from_dict({
            "model": model,
            "request": request
        }),
    )
)

# The response returns a work unit
work_unit = FunmanWorkUnit.from_dict(
    src_dict=json.loads(response.content.decode())
)
# Where the id can be used to pull ongoing results
work_unit.id

## Example 2: Step 3: Request results

In [ ]:
# Make a GET request to the API endpoint
response = asyncio.run(get_queries_queries_query_id_get.asyncio_detailed(
    work_unit.id,
    client=funman_client
))
# The response returns the latest funman results for the query
results = FunmanResults.from_dict(
    src_dict=json.loads(response.content.decode())
)
# pretty print the parameter_space
print(json.dumps(results.parameter_space.to_dict(), indent=2))

## Example 2: Step 4: Plot Results

In [ ]:
# Plot the ParameterSpace
from funman_demo.parameter_space_plotter import ParameterSpacePlotter
ParameterSpacePlotter(results.parameter_space).plot(show=True)

# Example 2: Optional: Halt Processing Job

In [ ]:
# Halt the current run
from funman_api_client.api.default import (
    halt_queries_query_id_halt_get
)

response = asyncio.run(
    halt_queries_query_id_halt_get.asyncio_detailed(
        client=funman_client,
        query_id=work_unit.id
    )
)

# The response returns a status
response